# Chapter 7: Ensemble Learning and Random Forests

---

## 📖 Rangkuman Chapter 7

Chapter ini membahas **Ensemble Learning**, sebuah teknik yang menggabungkan prediksi dari sekelompok prediktor (disebut *ensemble*) untuk mendapatkan prediksi yang lebih baik daripada prediktor individu terbaik.

Ide dasarnya mirip dengan **"wisdom of the crowd"**, di mana jawaban gabungan dari ribuan orang seringkali lebih baik daripada jawaban seorang ahli. Salah satu contoh paling terkenal adalah **Random Forest**, yang merupakan ensemble dari Decision Trees. Ensemble Learning sering digunakan untuk memenangkan kompetisi Machine Learning. Chapter ini akan mencakup metode ensemble populer: **bagging**, **boosting**, dan **stacking**.

## 🎯 Topics Covered

| No | Topic | Description |
|----|-------|-------------|
| 1 | **Voting Classifiers** | Menggabungkan prediksi dengan suara mayoritas (hard & soft voting) |
| 2 | **Bagging & Pasting** | Melatih prediktor pada subset acak data (dengan/tanpa replacement) |
| 3 | **Out-of-Bag (oob) Evaluation** | Mengevaluasi model bagging tanpa perlu validation set |
| 4 | **Random Forests** | Ensemble Decision Trees yang dilatih dengan metode bagging |
| 5 | **Extra-Trees** | Varian Random Forest yang lebih "acak" |
| 6 | **Feature Importance** | Cara Random Forest mengukur pentingnya setiap fitur |
| 7 | **Boosting (AdaBoost)** | Melatih prediktor secara berurutan, fokus pada kesalahan sebelumnya |
| 8 | **Gradient Boosting** | Melatih prediktor secara berurutan, fokus pada *residual errors* |
| 9 | **Stacking** | Menggunakan model (blender) untuk menggabungkan prediksi ensemble |

## 🗳️ Voting Classifiers

Ini adalah cara sederhana untuk membuat classifier yang lebih baik: latih beberapa classifier yang berbeda (misalnya, Logistic Regression, SVM, Random Forest), lalu gabungkan prediksi mereka.

* **Hard Voting Classifier**: Memprediksi kelas yang mendapatkan suara mayoritas (paling banyak dipilih) dari semua classifier. Anehnya, metode ini seringkali mencapai akurasi lebih tinggi daripada classifier terbaik di dalam ensemble.
* **Soft Voting Classifier**: Jika semua classifier dapat menghitung probabilitas (`predict_proba()`), Anda dapat memprediksi kelas dengan probabilitas rata-rata tertinggi. Ini seringkali berkinerja lebih baik karena memberi bobot lebih pada "suara" yang sangat percaya diri.

**Mengapa ini berhasil? (Law of Large Numbers)**
Anggap Anda memiliki 1.000 classifier "lemah" (weak learners) yang hanya 51% akurat (sedikit lebih baik dari tebakan acak). Jika Anda mengambil suara mayoritas, akurasi ensemble bisa mencapai 75%!

Ini berhasil dengan asumsi bahwa semua classifier **independen** dan membuat **kesalahan yang tidak berkorelasi**. Karena itu, metode ensemble bekerja paling baik ketika prediktornya **beragam (diverse)**.

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Load example dataset
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

log_clf = LogisticRegression(random_state=42)
rnd_clf = RandomForestClassifier(random_state=42)
svm_clf = SVC(random_state=42)

# Hard Voting
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard'
)
voting_clf.fit(X_train, y_train)

## 🛍️ Bagging and Pasting

Ini adalah pendekatan lain untuk mendapatkan classifier yang beragam. Alih-alih menggunakan algoritme yang berbeda, Anda menggunakan **algoritme yang sama** (misalnya, Decision Tree) tetapi melatihnya pada **subset acak yang berbeda** dari training set.

* **Bagging (Bootstrap Aggregating)**: Sampling dilakukan **dengan replacement** (`bootstrap=True`). Ini berarti satu instance bisa diambil beberapa kali untuk satu prediktor.
* **Pasting**: Sampling dilakukan **tanpa replacement** (`bootstrap=False`).

**Cara Kerja:**
Setelah semua prediktor dilatih (bisa secara paralel), ensemble membuat prediksi dengan mengambil **mode** (untuk klasifikasi) atau **rata-rata** (untuk regresi) dari semua prediksi individu.

**Hasil:** Ensemble memiliki **bias yang serupa** tetapi **variance yang lebih rendah** daripada satu prediktor yang dilatih pada semua data.

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Ensemble Bagging dengan 500 Decision Trees
# Setiap pohon dilatih pada 100 instance acak (dengan replacement)
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1
)
bag_clf.fit(X_train, y_train)

### Out-of-Bag (oob) Evaluation

Saat menggunakan bagging (sampling dengan replacement), rata-rata hanya **63%** instance pelatihan yang di-sampling untuk setiap prediktor. Instance yang tersisa (tidak terpilih) disebut **Out-of-Bag (oob)**.

Anda dapat mengevaluasi ensemble dengan menggunakan instance oob ini sebagai "validation set gratis" (`oob_score=True`). Ini memberikan estimasi performa model tanpa perlu validation set terpisah.

In [ ]:
bag_clf_oob = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True
)
bag_clf_oob.fit(X_train, y_train)
print(f"OOB Score: {bag_clf_oob.oob_score_}")

## 🌳 Random Patches & Random Subspaces

Bagging juga bisa mengacak fitur (bukan hanya instance).

* **Random Patches**: Sampling acak pada instance DAN fitur (`bootstrap=True`, `max_features < 1.0`, `bootstrap_features=True`).
* **Random Subspaces**: Sampling acak pada fitur saja (`bootstrap=False`, `max_samples=1.0`, `bootstrap_features=True`, `max_features < 1.0`).

Ini menambahkan lebih banyak keragaman (diversity), menukar bias yang sedikit lebih tinggi dengan variance lebih rendah. Berguna jika data memiliki banyak fitur (misalnya, gambar).

## 🌲 Random Forests

Random Forest adalah ensemble dari Decision Trees, yang dilatih menggunakan metode bagging (atau pasting), dengan `max_samples` = ukuran training set.

**Keunggulan:**
* Sangat cepat dan scalable.
* Mengukur **Feature Importance**: Menghitung seberapa banyak error meningkat ketika data untuk fitur tertentu diacak (permutation importance).

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

## 🎲 Extra-Trees

Extra-Trees (Extremely Randomized Trees) adalah varian dari Random Forest yang lebih acak.

* **Perbedaan Utama**: Saat membelah node, Extra-Trees menggunakan **threshold acak** (alih-alih mencari threshold optimal seperti Random Forest).
* **Keuntungan**: Lebih cepat dilatih (mencari threshold optimal memakan waktu).
* **Trade-off**: Menukar bias lebih tinggi dengan variance lebih rendah.

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

et_clf = ExtraTreesClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
et_clf.fit(X_train, y_train)

## 📊 Feature Importance

Random Forests bisa mengukur **pentingnya relatif** setiap fitur.

* **Cara Kerja**: Mengukur seberapa banyak error meningkat ketika data untuk fitur itu diacak.
* **Keuntungan**: Memberi wawasan cepat tentang fitur paling penting di dataset.

In [ ]:
# Contoh dengan dataset Iris
from sklearn.datasets import load_iris

iris = load_iris()
rnd_clf_iris = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf_iris.fit(iris["data"], iris["target"])

for name, score in zip(iris["feature_names"], rnd_clf_iris.feature_importances_):
    print(f"{name}: {score:.4f}")

## 🚀 Boosting

Boosting adalah metode ensemble sekuensial yang mencoba memperbaiki kesalahan dari prediktor sebelumnya.

### AdaBoost (Adaptive Boosting)

* **Cara Kerja**: Melatih prediktor pertama, lalu meningkatkan bobot instance yang *underfitted* (kesalahan prediksi). Prediktor kedua fokus lebih pada instance sulit itu, dan seterusnya.
* **Hyperparameters**: `n_estimators`, `learning_rate` (shrinkage).
* **Prediksi**: Rata-rata berbobot dari prediksi prediktor (prediktor lebih akurat punya bobot lebih tinggi).
* **SAMME & SAMME.R**: SAMME.R menggunakan probabilitas kelas (lebih baik).

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5
)
ada_clf.fit(X_train, y_train)

### Gradient Boosting

* **Cara Kerja**: Mirip AdaBoost, tapi fokus pada **residual errors** (selisih prediksi vs. target) dari prediktor sebelumnya.
* **Early Stopping**: Gunakan untuk mencari jumlah optimal pohon (`staged_predict()`).
* **Shrinkage**: Mengurangi kontribusi setiap pohon (`learning_rate` rendah = regularisasi).
* **Stochastic GB**: Subsampling instance (`subsample=0.25`) untuk kecepatan dan regularisasi.
* **XGBoost**: Implementasi GB yang dioptimasi, sangat cepat dan populer.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# GBRT dengan early stopping
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

## 🥞 Stacking

Stacking menggunakan **blender** (atau meta-learner) untuk mempelajari cara terbaik menggabungkan prediksi dari prediktor base.

* **Cara Kerja**: Bagi training set menjadi subset. Latih base prediktor pada subset 1, lalu gunakan prediksi mereka di subset 2 sebagai input untuk blender.
* **Multi-Layer Stacking**: Anda bisa punya beberapa layer blender.
* **Scikit-Learn**: Gunakan `StackingClassifier` atau implementasikan manual.

In [ ]:
from sklearn.ensemble import StackingClassifier

stacking_clf = StackingClassifier(
    estimators=[('lr', LogisticRegression()), ('rf', RandomForestClassifier()), ('svc', SVC())],
    final_estimator=RandomForestClassifier()
)
stacking_clf.fit(X_train, y_train)

## 🎯 Kesimpulan Chapter 7

### 📊 Summary Table

In [ ]:
print("="*80)
print("CHAPTER 7 SUMMARY - ENSEMBLE METHODS")
print("="*80)

summary_table = """
┌────────────────────┬──────────────────────┬────────────────────────────┐
│ Method             │ Key Idea             │ When to Use                │
├────────────────────┼──────────────────────┼────────────────────────────┤
│ Voting Classifier  │ Majority vote        │ Diverse classifiers        │
│                    │ (hard/soft)          │ Soft voting if possible    │
├────────────────────┼──────────────────────┼────────────────────────────┤
│ Bagging            │ Subset random        │ Reduce variance            │
│                    │ (w/ replacement)     │ Parallel training          │
├────────────────────┼──────────────────────┼────────────────────────────┤
│ Pasting            │ Subset random        │ Similar to bagging         │
│                    │ (w/o replacement)    │ Smaller datasets           │
├────────────────────┼──────────────────────┼────────────────────────────┤
│ Random Forest      │ Bagging + Trees      │ General purpose            │
│                    │ Random features      │ Feature importance         │
├────────────────────┼──────────────────────┼────────────────────────────┤
│ Extra-Trees        │ RF + Random splits   │ Faster training            │
│                    │                      │ Slightly higher bias       │
├────────────────────┼──────────────────────┼────────────────────────────┤
│ AdaBoost           │ Sequential           │ Weak learners              │
│                    │ Focus on errors      │ SAMME.R for probabilities  │
├────────────────────┼──────────────────────┼────────────────────────────┤
│ Gradient Boosting  │ Sequential           │ Trees as base              │
│                    │ Fit residuals        │ Early stopping             │
├────────────────────┼──────────────────────┼────────────────────────────┤
│ Stacking           │ Meta-learner         │ Complex ensembles          │
│                    │ (blender)            │ Multi-layer possible       │
└────────────────────┴──────────────────────┴────────────────────────────┘
"""

print(summary_table)

### 💡 Key Takeaways

**1. Ensemble > Single:**
- Gabungkan model lemah → model kuat.
- Keragaman (diversity) adalah kunci sukses.

**2. Bagging/ Random Forest:**
- Reduce variance dengan sampling acak.
- OOB untuk validasi gratis.
- Feature importance untuk insights.

**3. Boosting:**
- Sequential: Fokus pada kesalahan.
- AdaBoost: Adjust bobot instance.
- Gradient Boosting: Fit residuals.
- Gunakan shrinkage & early stopping.

**4. Stacking:**
- Gunakan blender untuk gabungkan prediksi.
- Lebih fleksibel untuk ensemble kompleks.

**5. Parallel vs Sequential:**
- Bagging/RF/Stacking: Parallel (cepat).
- Boosting: Sequential (lambat tapi kuat).

## 🔧 Exercises (from the book)

### Exercise 1
**Q:** If you have trained five different models on the exact same training data, and they all achieve 95% precision, is there any chance that you can combine these models to get better results?
**A:** Ya, ada peluang. Jika kelima model **membuat tipe kesalahan yang berbeda**, menggabungkan mereka dengan *voting classifier* (terutama soft voting) dapat meningkatkan akurasi/presisi. Jika mereka semua membuat kesalahan yang *sama persis*, menggabungkannya tidak akan membantu.

---

### Exercise 2
**Q:** What is the difference between hard and soft voting classifiers?
**A:**
* **Hard voting**: Menghitung suara dari setiap classifier dan memilih kelas yang paling banyak dipilih (mode statistik).
* **Soft voting**: Menghitung rata-rata probabilitas kelas yang diprediksi dari semua classifier, lalu memilih kelas dengan probabilitas rata-rata tertinggi. Soft voting seringkali lebih baik karena memberi bobot lebih pada suara yang sangat "percaya diri".

---

### Exercise 3
**Q:** Is it possible to speed up training of a bagging ensemble by distributing it across multiple servers? What about pasting, boosting, Random Forests, or stacking?
**A:**
* **Bagging:** **Ya**. Setiap prediktor dilatih secara independen pada subset data yang berbeda, sehingga dapat diparalelkan dengan sempurna.
* **Pasting:** **Ya**. Sama seperti bagging, pelatihan bersifat independen dan paralel.
* **Boosting:** **Tidak**. Boosting bersifat *sekuensial*; setiap prediktor baru bergantung pada hasil prediktor sebelumnya, sehingga tidak dapat diparalelkan.
* **Random Forests:** **Ya**. Sama seperti bagging, setiap pohon dilatih secara independen.
* **Stacking:** **Sebagian**. Prediktor-prediktor dalam *satu layer* dapat dilatih secara paralel. Namun, *antar layer* bersifat sekuensial (Layer 2 harus menunggu Layer 1 selesai, Blender harus menunggu Layer 1 selesai membuat prediksi).

---

### Exercise 4
**Q:** What is the benefit of out-of-bag evaluation?
**A:** Manfaat utamanya adalah Anda mendapatkan evaluasi model (mirip dengan cross-validation) secara **gratis** tanpa perlu membuat *validation set* terpisah. Ini karena setiap prediktor dievaluasi pada instance (oob) yang tidak pernah dilihatnya selama pelatihan.

---

### Exercise 5
**Q:** What makes Extra-Trees more random than regular Random Forests? How can this extra randomness help? Are Extra-Trees slower or faster?
**A:**
* **Lebih Acak**: Extra-Trees menggunakan *threshold* (ambang batas) acak untuk membelah fitur, sedangkan Random Forest mencari *threshold* optimal.
* **Manfaat**: Keacakan ekstra ini adalah bentuk *regularisasi* lain. Ini menukar bias yang sedikit lebih tinggi dengan variance yang lebih rendah.
* **Kecepatan**: Extra-Trees **lebih cepat** dilatih, karena mencari threshold optimal (yang dilakukan Random Forest) adalah salah satu tugas yang paling memakan waktu dalam menumbuhkan pohon.

---

### Exercise 6
**Q:** If your AdaBoost ensemble underfits the training data, which hyperparameters should you tweak and how?
**A:** Untuk mengurangi underfitting (meningkatkan kompleksitas):
1.  **`n_estimators`**: **Naikkan** jumlah estimator (pohon).
2.  **`learning_rate`**: **Naikkan** learning rate (default 1.0).
3.  **Regularisasi Base Estimator**: **Kurangi** regularisasi pada base estimator (misalnya, jika menggunakan `DecisionTreeClassifier`, *naikkan* `max_depth` dari 1).

---

### Exercise 7
**Q:** If your Gradient Boosting ensemble overfits the training set, should you increase or decrease the learning rate?
**A:** Anda harus **mengurangi** `learning_rate`. `learning_rate` yang lebih rendah (shrinkage) berarti setiap pohon memiliki kontribusi yang lebih kecil, yang meregularisasi model. Namun, Anda mungkin perlu *menaikkan* `n_estimators` (menggunakan early stopping) untuk mengimbanginya.

---

### Exercise 8 & 9
**Q:** (MNIST Stacking)
**A:** (Requires implementation)
* **Ex. 8:** Latih beberapa classifier (Random Forest, Extra-Trees, SVM) di MNIST. Gabungkan mereka menggunakan `VotingClassifier` (coba 'hard' dan 'soft'). Evaluasi di validation set, lalu test set, dan bandingkan hasilnya.
* **Ex. 9:** Gunakan classifier dari Ex. 8. Buat prediksi dari *validation set* untuk membuat *blending set* baru. Latih "blender" (misalnya, `RandomForestClassifier` lain) pada blending set ini. Evaluasi ensemble stacking ini di test set. Bandingkan hasilnya dengan Voting Classifier dari Ex. 8.

---

**Happy Ensembling! 🌲🌲🌲**